In [10]:
!pip install -q tensorflow
!pip install -q tensorflow-datasets
!pip install -q tensorflow-privacy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [12]:
import os

import tensorflow as tf
#tf.compat.v1.disable_v2_behavior()
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
#from keras import layers
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from sklearn.preprocessing import OneHotEncoder

from matplotlib import pyplot as plt

2023-04-16 11:32:01.963992: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-16 11:32:01.966911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 11:32:02.017271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 11:32:02.018243: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 11:32:05.436032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [13]:
ROOT_DIR = os.path.abspath('../')
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)
os.chdir(ROOT_DIR)

print(ROOT_DIR)

DATA_DIR = os.path.join(ROOT_DIR, 'data')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'models')

/usr4/ugrad/cjxu/Documents/MA751


In [168]:
print('Loading the dataset.')


(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

Loading the dataset.


In [169]:
encoder = OneHotEncoder(sparse = False)

y_train_onehot = encoder.fit_transform(y_train)
y_test_onehot = encoder.fit_transform(y_test)
print(y_train[0])
print(y_train_onehot[0])

[6]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [151]:
# # Load the CIFAR10 dataset
# print("Loading CIFAR10 dataset...")
# (ds_cifar10_train, ds_cifar10_test), ds_cifar10_info = tfds.load(
#         'cifar10',
#         split=['train', 'test'],
#         data_dir= DATA_DIR,
#         shuffle_files=True, # load in random order
#         as_supervised=True, # Include labels
#         with_info=True, # Include info
#     )

Loading CIFAR10 dataset...


In [152]:
# def normalize_img(image, label):
#     """Normalizes images: `uint8` -> `float32`."""
#     return tf.cast(image, tf.float32) / 255., label

In [153]:
# # Prepare cifar10 training dataset
# ds_cifar10_train = ds_cifar10_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
# ds_cifar10_train = ds_cifar10_train.cache()     # Cache data
# ds_cifar10_train = ds_cifar10_train.shuffle(ds_cifar10_info.splits['train'].num_examples)
# ds_cifar10_train = ds_cifar10_train.batch(64)  # <<<<< To change batch size, you have to change it here
# ds_cifar10_train = ds_cifar10_train.prefetch(tf.data.AUTOTUNE)

# # Prepare cifar10 test dataset
# ds_cifar10_test = ds_cifar10_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
# ds_cifar10_test = ds_cifar10_test.batch(64)    # <<<<< To change batch size, you have to change it here
# #ds_cifar10_test = ds_cifar10_test.cache()
# ds_cifar10_test = ds_cifar10_test.prefetch(tf.data.AUTOTUNE)

In [19]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1


In [170]:
tf.random.set_seed(0)

model = tf.keras.models.Sequential() 

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, kernel_size =(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation = "relu")) 
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = "relu")) 
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(rate=0.3))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation = "relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation = "relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(rate=0.4))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [147]:
"""
This part is to modify the optimizer for the DP library of the tensorflow because it cannot run on the tensorflow 2.0+. 
This is the recognition credit for the notebook:
https://colab.research.google.com/github/ricardocarvalhods/dpcgan/blob/master/DP_CGAN_MNIST.ipynb#scrollTo=hOBQELw7mo9Z&line=21&uniqifier=1
"""
from absl import logging
import collections

from tensorflow_privacy.privacy.dp_query import gaussian_query

def make_optimizer_class(cls):
  """Constructs a DP optimizer class from an existing one."""
  parent_code = tf.compat.v1.train.Optimizer.compute_gradients.__code__
  child_code = cls.compute_gradients.__code__
  GATE_OP = tf.compat.v1.train.Optimizer.GATE_OP  # pylint: disable=invalid-name
  if child_code is not parent_code:
    logging.warning(
        'WARNING: Calling make_optimizer_class() on class %s that overrides '
        'method compute_gradients(). Check to ensure that '
        'make_optimizer_class() does not interfere with overridden version.',
        cls.__name__)

  class DPOptimizerClass(cls):
    """Differentially private subclass of given class cls."""

    _GlobalState = collections.namedtuple(
      '_GlobalState', ['l2_norm_clip', 'stddev'])
    
    def __init__(
        self,
        dp_sum_query,
        num_microbatches=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg, g-doc-args
        **kwargs):
      """Initialize the DPOptimizerClass.

      Args:
        dp_sum_query: DPQuery object, specifying differential privacy
          mechanism to use.
        num_microbatches: How many microbatches into which the minibatch is
          split. If None, will default to the size of the minibatch, and
          per-example gradients will be computed.
        unroll_microbatches: If true, processes microbatches within a Python
          loop instead of a tf.while_loop. Can be used if using a tf.while_loop
          raises an exception.
      """
      super(DPOptimizerClass, self).__init__(*args, **kwargs)
      self._dp_sum_query = dp_sum_query
      self._num_microbatches = num_microbatches
      self._global_state = self._dp_sum_query.initial_global_state()
      # TODO(b/122613513): Set unroll_microbatches=True to avoid this bug.
      # Beware: When num_microbatches is large (>100), enabling this parameter
      # may cause an OOM error.
      self._unroll_microbatches = unroll_microbatches

    def compute_gradients(self,
                          loss,
                          var_list,
                          gate_gradients=GATE_OP,
                          aggregation_method=None,
                          colocate_gradients_with_ops=False,
                          grad_loss=None,
                          gradient_tape=None,
                          curr_noise_mult=0,
                          curr_norm_clip=1):

      self._dp_sum_query = gaussian_query.GaussianSumQuery(curr_norm_clip, 
                                                           curr_norm_clip*curr_noise_mult)
      self._global_state = self._dp_sum_query.make_global_state(curr_norm_clip, 
                                                                curr_norm_clip*curr_noise_mult)
      

      # TF is running in Eager mode, check we received a vanilla tape.
      if not gradient_tape:
        raise ValueError('When in Eager mode, a tape needs to be passed.')

      vector_loss = loss()
      if self._num_microbatches is None:
        self._num_microbatches = tf.shape(input=vector_loss)[0]
      sample_state = self._dp_sum_query.initial_sample_state(var_list)
      microbatches_losses = tf.reshape(vector_loss, [self._num_microbatches, -1])
      sample_params = (self._dp_sum_query.derive_sample_params(self._global_state))

      def process_microbatch(i, sample_state):
        """Process one microbatch (record) with privacy helper."""
        microbatch_loss = tf.reduce_mean(input_tensor=tf.gather(microbatches_losses, [i]))
        grads = gradient_tape.gradient(microbatch_loss, var_list)
        sample_state = self._dp_sum_query.accumulate_record(sample_params, sample_state, grads)
        return sample_state
    
      for idx in range(self._num_microbatches):
        sample_state = process_microbatch(idx, sample_state)

      if curr_noise_mult > 0:
        grad_sums, self._global_state = (self._dp_sum_query.get_noised_result(sample_state, self._global_state))
      else:
        grad_sums = sample_state

      def normalize(v):
        return v / tf.cast(self._num_microbatches, tf.float32)

      final_grads = tf.nest.map_structure(normalize, grad_sums)
      grads_and_vars = final_grads#list(zip(final_grads, var_list))
    
      return grads_and_vars

  return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
  """Constructs a DP optimizer with Gaussian averaging of updates."""

  class DPGaussianOptimizerClass(make_optimizer_class(cls)):
    """DP subclass of given class cls using Gaussian averaging."""

    def __init__(
        self,
        l2_norm_clip,
        noise_multiplier,
        num_microbatches=None,
        ledger=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg
        **kwargs):
      dp_sum_query = gaussian_query.GaussianSumQuery(
          l2_norm_clip, l2_norm_clip * noise_multiplier)

      if ledger:
        dp_sum_query = privacy_ledger.QueryWithLedger(dp_sum_query,
                                                      ledger=ledger)

      super(DPGaussianOptimizerClass, self).__init__(
          dp_sum_query,
          num_microbatches,
          unroll_microbatches,
          *args,
          **kwargs)

    @property
    def ledger(self):
      return self._dp_sum_query.ledger

  return DPGaussianOptimizerClass

In [171]:
GradientDescentOptimizer = tf.compat.v1.train.GradientDescentOptimizer
DPAdamGaussianOptimizer_new = make_gaussian_optimizer_class(tf.compat.v1.train.AdamOptimizer)

In [172]:
learning_rate=0.001
l2_norm_clip = 1.5
batch_size = 64
num_microbatches=batch_size
noise_multiplier = 1.0

optimizer = DPAdamGaussianOptimizer_new(
    learning_rate=learning_rate,
    l2_norm_clip = l2_norm_clip,
    num_microbatches=num_microbatches,
    noise_multiplier = noise_multiplier)

model.compile(optimizer = optimizer, 
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [167]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_60 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_61 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 batch_normalization_61 (Bat  (None, 30, 30, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                     

In [173]:
model.fit(x_train,
          y_train_onehot, 
          batch_size = 64 , 
          epochs = 100, 
          validation_data = (x_test,y_test_onehot))

Epoch 1/100


/usr4/ugrad/cjxu/.local/lib/python3.10/site-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


782/782 [==============================] - 17s 21ms/step - loss: 1.7834 - categorical_accuracy: 0.3682 - val_loss: 1.3424 - val_categorical_accuracy: 0.5163
Epoch 2/100
782/782 [==============================] - 16s 20ms/step - loss: 1.3100 - categorical_accuracy: 0.5378 - val_loss: 1.0520 - val_categorical_accuracy: 0.6256
Epoch 3/100
782/782 [==============================] - 16s 20ms/step - loss: 1.0929 - categorical_accuracy: 0.6174 - val_loss: 0.9372 - val_categorical_accuracy: 0.6738
Epoch 4/100
782/782 [==============================] - 16s 20ms/step - loss: 0.9571 - categorical_accuracy: 0.6676 - val_loss: 0.9466 - val_categorical_accuracy: 0.6704
Epoch 5/100
782/782 [==============================] - 16s 20ms/step - loss: 0.8728 - categorical_accuracy: 0.7007 - val_loss: 0.7783 - val_categorical_accuracy: 0.7298
Epoch 6/100
782/782 [==============================] - 16s 20ms/step - loss: 0.8029 - categorical_accuracy: 0.7265 - val_loss: 0.8016 - val_categorical_accuracy: 0.731